In [1]:
## Advanced Lane Finding Project

''''The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.'''

##---
## First, I'll compute the camera calibration using chessboard image

'\'The goals / steps of this project are the following:\n\n* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.\n* Apply a distortion correction to raw images.\n* Use color transforms, gradients, etc., to create a thresholded binary image.\n* Apply a perspective transform to rectify binary image ("birds-eye view").\n* Detect lane pixels and fit to find the lane boundary.\n* Determine the curvature of the lane and vehicle position with respect to center.\n* Warp the detected lane boundaries back onto the original image.\n* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.'

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import math
%matplotlib inline


def Calc_CalibMatrix():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        #plt.imshow(img)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img1 = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #plt.imshow(img1)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    #dst = cv2.undistort(img, mtx, dist, None, mtx)
    return mtx,dist

def undistort_img(image,mtx_arg,dist_arg):
    undist_img = cv2.undistort(image, mtx_arg, dist_arg, None, mtx_arg)
    return undist_img


In [3]:
def hls_Sspace(img, thresh=(0, 255)):
    # Convert to HLS color space
    hls_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # Apply a threshold to the S channel
    S_img = hls_img[:,:,2]
    # Return a binary image of threshold result
    S_bin = np.zeros_like(S_img)
    S_bin[(S_img > thresh[0]) & (S_img <= thresh[1])] = 1
    sSpace_binary_output = np.copy(S_bin) # placeholder line
    
    return sSpace_binary_output

In [4]:
def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Take the derivative in x or y given orient = 'x' or 'y'
    ## only take the one required to improve performance
    if orient == 'x':
        sobel = cv2.Sobel(gray_img, cv2.CV_64F,1,0, ksize=sobel_kernel)
    elif orient == 'y':
        sobel = cv2.Sobel(gray_img, cv2.CV_64F,0,1, ksize=sobel_kernel)
    else:
        print("Error: Wrong Orientation") #To be changed to Error Handler
        
    # Get the absolute value of the sobel
    sobel_dir_abs = np.absolute(sobel)
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*sobel_dir_abs/np.max(sobel_dir_abs))
    # Create a binary mask 
    sobel_binary = np.zeros_like(scaled_sobel)
    # Make 1's where the sobel is within the thresholds 
    sobel_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    # Return
    grad_binary = np.copy(sobel_binary)
    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Get the gradient in x and y
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1,0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0,1, ksize=sobel_kernel)
    # Calculate the magnitude 
    sobel_mag = np.sqrt(sobelx**2 + sobely**2)
    # Scale to 8-bit and convert to uint8 type
    sobel_scaled = np.uint8(255*sobel_mag/np.max(sobel_mag))
    # Create a binary mask 
    sobel_mag_binary = np.zeros_like(sobel_scaled)
    # Make 1's where the sobel_mag is within the thresholds 
    sobel_mag_binary[(sobel_scaled >= mag_thresh[0]) & (sobel_scaled <= mag_thresh[1])] = 1
    
    # Return
    mag_binary = sobel_mag_binary
    return mag_binary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Convert to grayscale
    gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Get the gradient in x and y
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1,0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0,1, ksize=sobel_kernel)
    # Get the absolute value of the x and y gradients
    sobelx_abs = np.absolute(sobelx)
    sobely_abs = np.absolute(sobely)
    # Get the direction of the gradient 
    grad = np.arctan2(sobely_abs, sobelx_abs)
    # Create a binary
    sobel_binary = np.zeros_like(gray_img)
    # Make 1's where the sobel_dir is within the thresholds 
    sobel_binary[(grad >= thresh[0]) & (thresh[1] >= grad)] = 1
    
    # Return
    dir_binary = sobel_binary
    return dir_binary


def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [5]:

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom quater of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//4:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 72
    # Set the width of the windows +/- margin
    margin = 50
    # Set minimum number of pixels found to recenter window
    minpix = 100

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        # Update the boundaries of the window
        win_xleft_low = leftx_current - margin  
        win_xleft_high = leftx_current + margin 
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        #(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),
        #(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # Recenter next window on their mean position 
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def visualize_lane(binary_warped, leftx, lefty, rightx, righty, left_fitx, right_fitx, y_values):
    # Visualization
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255

    # Color in left and right line pixels
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    # cv2.fillPoly(out_img, np.int_([line_pts]), (0,255, 0))

    lane_img = np.zeros_like(out_img)
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line = np.array([np.flipud(np.transpose(np.vstack([left_fitx, y_values])))])
    right_line = np.array([np.transpose(np.vstack([right_fitx, y_values]))])
    lane_pts = np.hstack((left_line, right_line))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(lane_img, np.int_([lane_pts]), (0, 255, 0))
    out_img = cv2.addWeighted(out_img, 1, lane_img, 0.6, 0)

    return out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Get second order polynomial for the found coordinates
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    #print(binary_warped.shape[0])
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left_fitx` and `right_fitx` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ## 
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    #print(leftx.shape)
    #print(rightx.shape)
    #print(ploty.shape)
    #left_line = np.array([np.flipud(np.transpose(np.vstack([left_fitx, ploty])))])
    #right_line = np.array([np.transpose(np.vstack([rightx, ploty]))])
    #line_pts = np.hstack((left_line, right_line))
    
    # Color in left and right line pixels
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    #cv2.fillPoly(window_img, np.int_([line_pts]), (0,255, 0))
    
    out_img = visualize_lane(binary_warped, leftx, lefty, rightx, righty, left_fitx, right_fitx, ploty)
    
    return out_img, left_fit, right_fit, ploty, leftx, rightx

def measure_curvature(ploty, left_fit_cr, right_fit_cr, xm, ym):
    # Define conversions in x and y from pixels space to meters
    #ym_per_pix = 30/720 # meters per pixel in y dimension
    #xm_per_pix = 3.7/800 # meters per pixel in x dimension
      
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve
    left_curverad = (((1+(2*left_fit_cr[0]*y_eval*ym + left_fit_cr[1])**2)**(3/2)) / np.abs(2* left_fit_cr[0]))
    right_curverad = (((1+(2*right_fit_cr[0]*y_eval*ym + right_fit_cr[1])**2)**(3/2)) / np.abs(2* right_fit_cr[0]))
    
    return left_curverad, right_curverad

In [6]:
im_test = cv2.imread('./test_images/test1.jpg')
img_size = (im_test.shape[1], im_test.shape[0])
## For the Image Transformation
### Source Points of the image
### Assumption that the camera is in the middle of the car
src = np.float32(
        [[(img_size[0]//2)-500, img_size[1]],
        [((img_size[0]//2)-100), 480],
        [((img_size[0]//2)+100), 480],
        [(img_size[0]//2)+500, img_size[1]]])

### Destination Points of the image
dst = np.float32(
        [[200,img_size[1]],
        [200, 0],
        [img_size[0]-200, 0],
        [img_size[0]-200, img_size[1]]])

## Area of Interest
### Region for the lane marking area:
region = np.array([[(180,img_size[1]),(580,450), (720,450), (1200,img_size[1])]], dtype=np.int)



## Parametes for Sobel
sX_kernel = 31
sY_kernel = 31
sMag_kernel = 27
sDir_kernel = 31
sX_thresh = (70, 255)
sY_thresh = (70, 255)
sMag_thresh = (100, 255)
sDir_thresh = (0.85, 1.05)
hls_thresh = (100, 255)

## Parameters for real world transfrom
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/800 # meters per pixel in x dimension

In [7]:
mtx_camera, dist_camera = Calc_CalibMatrix()

## Get Image and Undistrot Image
img = cv2.imread('./test_images/straight_lines1.jpg')
img_undist = undistort_img(img, mtx_camera, dist_camera)


In [8]:
result_x = abs_sobel_thresh(img_undist, 'x', sX_kernel, sX_thresh) 
result_y = abs_sobel_thresh(img_undist, 'y', sY_kernel, sY_thresh)
result_mag = mag_thresh(img_undist, sMag_kernel, sMag_thresh)
result_dir = dir_threshold(img_undist, sDir_kernel, sDir_thresh)
result_hls = hls_Sspace(img_undist, hls_thresh)

combined = np.zeros_like(result_x)
combined[((result_x == 1) & (result_y == 1)) | (((result_mag == 1) & (result_dir == 1))) | (result_hls == 1)] = 1
combined_img = region_of_interest(combined, region)

In [9]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(image):  
    ## Get Image and Undistrot Image
    #img = np.copy(image) 
    img_undist = undistort_img(image, mtx_camera, dist_camera)
    region = np.array([[(180, img_undist.shape[0]), (580, 450), (720, 450), (1200, img_undist.shape[0])]], dtype=np.int)
    ## Create a binary Image
    result_x = abs_sobel_thresh(img_undist, 'x', sX_kernel, sX_thresh) 
    result_y = abs_sobel_thresh(img_undist, 'y', sY_kernel, sY_thresh)
    result_mag = mag_thresh(img_undist, sMag_kernel, sMag_thresh)
    result_dir = dir_threshold(img_undist, sDir_kernel, sDir_thresh)
    result_hls = hls_Sspace(img_undist, hls_thresh)

    combined = np.zeros_like(result_x)
    combined[((result_x == 1) & (result_y == 1)) | (((result_mag == 1) & (result_dir == 1))) | (result_hls == 1)] = 1
    combined_img = region_of_interest(combined, region)

    ### Create Image Size Parameters for warpPerspective
    img_size = (img_undist.shape[1], img_undist.shape[0])

    mtx_persp = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(combined_img, mtx_persp, img_size, flags=cv2.INTER_LINEAR)

    ## Detect the Lines in the warped Image
    ### Get the Polynomiyal for the left and right side
    ### also get the amount of pixels detected to use later as a weighting.
    ### The more pixles detected the higher the accuracy.
    out_img, left_poly, right_poly, poly, leftx, rightx = fit_polynomial(warped)

    ## Getting Curvature for left and right side
    left_r, right_r = measure_curvature(poly, left_poly, right_poly, xm_per_pix, ym_per_pix)

    ## Weight the curvature ratios based on the amount of pixles
    weighted_r = (((left_r * leftx.shape[0]) + (right_r * rightx.shape[0])) / (((leftx.shape[0] + rightx.shape[0]))))
    weighted_r = math.trunc(weighted_r * 100) / 100
    radius_str = "The Curve radius is: " + str(weighted_r) + 'm'

    ## Get the position and offset in the lane
    position = (np.average(leftx) + np.average(rightx)) / 2
    offset = ((img_size[0]/2)-position)*xm_per_pix
    offset = math.trunc(offset * 100) / 100
    offset_str = "Offset: " + str(offset) + 'm'

    ## Print the lines back into the image:
    font = cv2.FONT_HERSHEY_SIMPLEX
    mtx_persp_inv = cv2.getPerspectiveTransform(dst, src)
    lines_img = cv2.warpPerspective(out_img, mtx_persp_inv, img_size, flags=cv2.INTER_LINEAR)
    output_img = cv2.addWeighted(img_undist, 1, lines_img, 0.9, 0.2)
    cv2.putText(output_img,radius_str,(10,50), font, 1.2,(255,255,255),2, cv2.LINE_AA)
    cv2.putText(output_img,offset_str,(10,100), font, 1.2,(255,255,255),2, cv2.LINE_AA)
    
    return output_img

In [10]:
mtx_camera, dist_camera = Calc_CalibMatrix()
white_output = 'Output_project_video.mp4'
clip1 = VideoFileClip("./project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video Output_project_video.mp4
[MoviePy] Writing video Output_project_video.mp4


100%|█████████▉| 1260/1261 [11:00<00:00,  1.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Output_project_video.mp4 

CPU times: user 7min 57s, sys: 1min 4s, total: 9min 2s
Wall time: 11min 3s
